# This project is House price prediction in Tehran, based on data collected from Divar application.We have to train a model with features if the dataset and then try to caclulate the price of houses with new data

In [73]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from pycaret.regression import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

import pickle


In [74]:
df = pd.read_csv(r'C:\Users\Sazgar\Desktop\DataSets\tehranhouses.csv')
df.head(10)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1850000000,61666.67
1,60,1,True,True,True,Shahran,1850000000,61666.67
2,79,2,True,True,True,Pardis,550000000,18333.33
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33
5,70,2,True,True,False,North Program Organization,2050000000,68333.33
6,87,2,True,True,True,Pardis,600000000,20000.00
7,59,1,True,True,True,Shahran,2150000000,71666.67
8,54,2,True,True,False,Andisheh,493000000,16433.33
9,71,1,True,True,True,West Ferdows Boulevard,2370000000,79000.00


# To understand the relationship between fetures and price, scatter plot is used

In [75]:
fig = px.scatter(df, x = "Price", y = "Area", color = 'Parking',
                 size = 'Room', facet_col = 'Elevator', facet_row= "Parking")
fig.show()

In [76]:
fig_room = px.scatter(df, x ='Price', y = 'Room', facet_col = 'Parking', facet_row = 'Elevator', size = 'Price')
fig_room.show()

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        3479 non-null   object 
 1   Room        3479 non-null   int64  
 2   Parking     3479 non-null   bool   
 3   Warehouse   3479 non-null   bool   
 4   Elevator    3479 non-null   bool   
 5   Address     3456 non-null   object 
 6   Price       3479 non-null   int64  
 7   Price(USD)  3479 non-null   float64
dtypes: bool(3), float64(1), int64(2), object(2)
memory usage: 146.2+ KB


# After years price is changed based on dollar rate in Iran, to have updated price I decided to have new prices based the dollar value in Iran at the time

In [78]:
#After years price is changed based on dollar rate in Iran, to have updated price 
#I decided to have new prices based the dollar value in Iran at the time
df['Updated_Price'] = df['Price(USD)']*60000

In [79]:
df.head(10)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD),Updated_Price
0,63,1,True,True,True,Shahran,1850000000,61666.67,3.700000e+09
1,60,1,True,True,True,Shahran,1850000000,61666.67,3.700000e+09
2,79,2,True,True,True,Pardis,550000000,18333.33,1.100000e+09
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33,1.805000e+09
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33,1.400000e+10
5,70,2,True,True,False,North Program Organization,2050000000,68333.33,4.100000e+09
6,87,2,True,True,True,Pardis,600000000,20000.00,1.200000e+09
7,59,1,True,True,True,Shahran,2150000000,71666.67,4.300000e+09
8,54,2,True,True,False,Andisheh,493000000,16433.33,9.859998e+08
9,71,1,True,True,True,West Ferdows Boulevard,2370000000,79000.00,4.740000e+09


# By deviding these values to a billion, it became easier to read

In [80]:

df['Price(B)'] = (df['Updated_Price']/1000000000).round(3)

In [81]:
df.drop(columns = ['Price(USD)','Price', 'Updated_Price'], inplace =True)

In [82]:
df.head(5)

,Area,Room,Parking,Warehouse,Elevator,Address,Price(B)
0,63,1,True,True,True,Shahran,3.700
1,60,1,True,True,True,Shahran,3.700
2,79,2,True,True,True,Pardis,1.100
3,95,2,True,True,True,Shahrake Qods,1.805
4,123,2,True,True,True,Shahrake Gharb,14.000


# The area type was object because of ',' in it, by the code below, I turn this feature into 'int64'

In [83]:
df['Area'] = df['Area'].apply(lambda x: x.replace(',', ''))

In [84]:
df['Area'] = pd.to_numeric(df.Area, errors = 'coerce')

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3479 non-null   int64  
 1   Room       3479 non-null   int64  
 2   Parking    3479 non-null   bool   
 3   Warehouse  3479 non-null   bool   
 4   Elevator   3479 non-null   bool   
 5   Address    3456 non-null   object 
 6   Price(B)   3479 non-null   float64
dtypes: bool(3), float64(1), int64(2), object(1)
memory usage: 119.0+ KB


# To train our model we need to have all the features in numeric type


In [86]:
booleans = ['Parking', 'Warehouse', 'Elevator']
df[booleans] = df[booleans].astype(int)

In [87]:
df

,Area,Room,Parking,Warehouse,Elevator,Address,Price(B)
0,63,1,1,1,1,Shahran,3.700
1,60,1,1,1,1,Shahran,3.700
2,79,2,1,1,1,Pardis,1.100
3,95,2,1,1,1,Shahrake Qods,1.805
4,123,2,1,1,1,Shahrake Gharb,14.000
...,...,...,...,...,...,...,...
3474,86,2,1,1,1,Southern Janatabad,7.000
3475,83,2,1,1,1,Niavaran,13.600
3476,75,2,0,0,0,Parand,0.730
3477,105,2,1,1,1,Dorous,11.200


In [88]:
df.isna().sum()

Area          0
Room          0
Parking       0
Warehouse     0
Elevator      0
Address      23
Price(B)      0
dtype: int64

In [89]:
df.dropna(inplace = True)

# Using pycaret as an auto ML module to find best model

In [90]:
py_reg = setup(df, target='Price(B)')
best_model = compare_models()

,Description,Value
0,Session id,1071
1,Target,Price(B)
2,Target type,Regression
3,Original data shape,"(3456, 7)"
4,Transformed data shape,"(3456, 7)"
5,Transformed train set shape,"(2419, 7)"
6,Transformed test set shape,"(1037, 7)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,2.9317,70.6298,8.0085,0.7434,0.3528,0.5529,0.0480
lightgbm,Light Gradient Boosting Machine,2.9811,76.8542,8.3367,0.7279,0.3490,0.7881,0.0690
gbr,Gradient Boosting Regressor,3.2528,83.6366,8.8003,0.6907,0.3823,1.5986,0.0270
rf,Random Forest Regressor,3.0677,86.9981,8.8279,0.6875,0.3575,0.9640,0.0550
knn,K Neighbors Regressor,3.6006,99.9935,9.5605,0.6445,0.4176,1.3256,0.0140
ada,AdaBoost Regressor,6.2362,114.3570,10.2455,0.5872,0.7854,2.7694,0.0220
dt,Decision Tree Regressor,3.5755,123.2488,10.7362,0.5231,0.4301,0.9408,0.0130
br,Bayesian Ridge,6.0394,143.4160,11.5587,0.4878,0.6545,1.5060,0.0130
ridge,Ridge Regression,6.0566,143.3855,11.5601,0.4876,0.6554,1.5140,0.0140
lr,Linear Regression,6.0587,143.3851,11.5604,0.4875,0.6555,1.5150,0.7930


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [91]:
df.describe()

,Area,Room,Parking,Warehouse,Elevator,Price(B)
count,3.456000e+03,3456.000000,3456.000000,3456.000000,3456.000000,3456.000000
mean,8.802191e+06,2.081308,0.847512,0.914931,0.787326,10.759913
std,3.177783e+08,0.759723,0.359546,0.279025,0.409258,16.243459
min,3.000000e+01,0.000000,0.000000,0.000000,0.000000,0.007000
25%,6.900000e+01,2.000000,1.000000,1.000000,1.000000,2.840000
50%,9.000000e+01,2.000000,1.000000,1.000000,1.000000,5.800000
75%,1.210000e+02,2.000000,1.000000,1.000000,1.000000,12.000000
max,1.616000e+10,5.000000,1.000000,1.000000,1.000000,184.800000


In [92]:
df['Area'].unique()

array([         63,          60,          79,          95,         123,
                70,          87,          59,          54,          71,
                68,          64,         136,         155,         140,
                42,          93,          65,          99,         105,
               160,          77,         110,         100,          90,
                49,          96,          67,          62,          55,
               129,         109,          58,         150,         130,
                88,          51,         113,          98,          75,
                61,          72,         122,         215,         101,
                53,          74,         114,         151,         300,
                76,         148,          40,         128,          94,
                97,         137,          85,          78,          48,
                82,         120,         139,          66,          80,
                44,          50,         121,         141,      

# 4 values in Area are out of logical range which should be deleted

In [93]:
unnormal_areas = [8400000000, 3310000000, 16160000000, 2550000000]
df = df[~df['Area'].isin(unnormal_areas)]

In [94]:
df.describe()

,Area,Room,Parking,Warehouse,Elevator,Price(B)
count,3452.000000,3452.000000,3452.000000,3452.000000,3452.000000,3452.000000
mean,108.188007,2.081112,0.847335,0.914832,0.787080,10.754583
std,92.734668,0.759998,0.359717,0.279172,0.409431,16.247893
min,30.000000,0.000000,0.000000,0.000000,0.000000,0.007000
25%,69.000000,2.000000,1.000000,1.000000,1.000000,2.839500
50%,90.000000,2.000000,1.000000,1.000000,1.000000,5.800000
75%,120.000000,2.000000,1.000000,1.000000,1.000000,12.000000
max,3600.000000,5.000000,1.000000,1.000000,1.000000,184.800000


# Outliers can led the model to bias, so we have to detect and remove them

In [96]:
px.box(x = df['Area'], data_frame = df)

In [97]:
px.box(x = df['Price(B)'], data_frame = df)

In [98]:
def iqr (col):
    sorted(col)
    q1, q3 = col.quantile([0.25, 0.75])
    IQR = q3 - q1
    lwr_b = q1 - (1.5*IQR)
    upr_b = q3 + (1.5*IQR)
    return lwr_b, upr_b

In [99]:
low, high = iqr(df['Area'])
print(low, high)

-7.5 196.5


In [100]:
df['Area'] = np.where(df['Area']>high, high, df['Area'])
df['Area'] = np.where(df['Area']<low, low, df['Area'])

In [101]:
low, high = iqr(df['Price(B)'])
print(low, high)

-10.901249999999997 25.74075


In [102]:
df['Price(B)'] = np.where(df['Price(B)']>high, high, df['Price(B)'])
df['Price(B)'] = np.where(df['Price(B)']<low, low, df['Price(B)'])

In [103]:
px.box(x = df['Area'], data_frame = df)

In [104]:
px.box(x = df['Price(B)'], data_frame = df)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3452 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3452 non-null   float64
 1   Room       3452 non-null   int64  
 2   Parking    3452 non-null   int32  
 3   Warehouse  3452 non-null   int32  
 4   Elevator   3452 non-null   int32  
 5   Address    3452 non-null   object 
 6   Price(B)   3452 non-null   float64
dtypes: float64(2), int32(3), int64(1), object(1)
memory usage: 175.3+ KB


In [106]:
ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')

In [107]:
ndf = ohe.fit_transform(df[['Address']])

In [108]:
ndf

,Address_Abazar,Address_Abbasabad,Address_Absard,Address_Abuzar,Address_Afsarieh,Address_Ahang,Address_Air force,Address_Ajudaniye,Address_Alborz Complex,Address_Aliabad South,...,Address_Waterfall,Address_West Ferdows Boulevard,Address_West Pars,Address_Yaftabad,Address_Yakhchiabad,Address_Yousef Abad,Address_Zafar,Address_Zaferanieh,Address_Zargandeh,Address_Zibadasht
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
for col in ndf.columns:
    if col.startswith('Address_'):
        new_col_name = col.replace('Address_', '')
        ndf.rename(columns = {col:new_col_name}, inplace = True)

In [110]:
df = df.drop(columns='Address', axis=1)  
df = df.reset_index()  
ndf = ndf.reset_index() 
concat_df = pd.concat([df, ndf], axis=True)  

In [111]:
concat_df = concat_df.drop(columns = 'index', axis = 1)

In [112]:
concat_df

,Area,Room,Parking,Warehouse,Elevator,Price(B),Abazar,Abbasabad,Absard,Abuzar,...,Waterfall,West Ferdows Boulevard,West Pars,Yaftabad,Yakhchiabad,Yousef Abad,Zafar,Zaferanieh,Zargandeh,Zibadasht
0,63.0,1,1,1,1,3.700,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,60.0,1,1,1,1,3.700,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,79.0,2,1,1,1,1.100,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,95.0,2,1,1,1,1.805,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,123.0,2,1,1,1,14.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,86.0,2,1,1,1,7.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3448,83.0,2,1,1,1,13.600,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3449,75.0,2,0,0,0,0.730,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3450,105.0,2,1,1,1,11.200,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Outliers detected and removed and use the pycaret to see the changes

In [113]:

py_reg1 = setup(concat_df, target='Price(B)')

,Description,Value
0,Session id,4385
1,Target,Price(B)
2,Target type,Regression
3,Original data shape,"(3452, 198)"
4,Transformed data shape,"(3452, 198)"
5,Transformed train set shape,"(2416, 198)"
6,Transformed test set shape,"(1036, 198)"
7,Numeric features,197
8,Preprocess,True
9,Imputation type,simple


In [114]:
best_models1 = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,1.7149,6.6137,2.5564,0.8887,0.3283,0.4340,0.0300
ridge,Ridge Regression,1.7281,6.6804,2.5680,0.8876,0.3329,0.4409,0.0250
rf,Random Forest Regressor,1.6086,8.6206,2.9200,0.8545,0.3126,0.3277,0.1610
et,Extra Trees Regressor,1.5651,8.8575,2.9627,0.8506,0.3099,0.3029,0.2040
gbr,Gradient Boosting Regressor,1.9782,9.0692,2.9941,0.8471,0.3583,0.4633,0.1030
huber,Huber Regressor,2.0949,9.4853,3.0562,0.8402,0.3952,0.5552,0.4400
lightgbm,Light Gradient Boosting Machine,1.9609,10.6454,3.2298,0.8211,0.3556,0.4170,0.1050
omp,Orthogonal Matching Pursuit,2.2798,11.0558,3.3025,0.8141,0.4077,0.5883,0.0250
dt,Decision Tree Regressor,1.7966,12.1130,3.4618,0.7958,0.3693,0.3510,0.0300
knn,K Neighbors Regressor,2.5352,16.6662,4.0584,0.7180,0.4581,0.6062,0.0380


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [115]:
rs = RobustScaler()
cols = concat_df.columns
scaled_df = pd.DataFrame(rs.fit_transform(concat_df), columns = cols)

In [116]:
scaled_df

,Area,Room,Parking,Warehouse,Elevator,Price(B),Abazar,Abbasabad,Absard,Abuzar,...,Waterfall,West Ferdows Boulevard,West Pars,Yaftabad,Yakhchiabad,Yousef Abad,Zafar,Zaferanieh,Zargandeh,Zibadasht
0,-0.529412,-1.0,0.0,0.0,0.0,-0.229245,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.588235,-1.0,0.0,0.0,0.0,-0.229245,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.215686,0.0,0.0,0.0,0.0,-0.513072,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.098039,0.0,0.0,0.0,0.0,-0.436112,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.647059,0.0,0.0,0.0,0.0,0.895148,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,-0.078431,0.0,0.0,0.0,0.0,0.130997,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3448,-0.137255,0.0,0.0,0.0,0.0,0.851482,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3449,-0.294118,0.0,-1.0,-1.0,-1.0,-0.553463,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3450,0.294118,0.0,0.0,0.0,0.0,0.589487,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Final auot ML was need to see which model presents better results

In [117]:
py_reg1 = setup(scaled_df, target='Price(B)')
best_models2 = compare_models()

,Description,Value
0,Session id,1610
1,Target,Price(B)
2,Target type,Regression
3,Original data shape,"(3452, 198)"
4,Transformed data shape,"(3452, 198)"
5,Transformed train set shape,"(2416, 198)"
6,Transformed test set shape,"(1036, 198)"
7,Numeric features,197
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.1874,0.0834,0.2866,0.8794,0.1412,0.9040,0.0420
ridge,Ridge Regression,0.1884,0.0841,0.2878,0.8784,0.1409,0.9067,0.0270
huber,Huber Regressor,0.1839,0.0852,0.2896,0.8768,0.1427,0.8255,0.4380
knn,K Neighbors Regressor,0.1730,0.0953,0.3051,0.8628,0.1493,0.8056,0.0570
rf,Random Forest Regressor,0.1747,0.1010,0.3138,0.8526,0.1559,0.8860,0.1790
et,Extra Trees Regressor,0.1726,0.1048,0.3202,0.8472,0.1550,0.8780,0.2170
gbr,Gradient Boosting Regressor,0.2162,0.1095,0.3291,0.8405,0.1657,1.0372,0.1100
lightgbm,Light Gradient Boosting Machine,0.2125,0.1237,0.3488,0.8211,0.1713,0.9493,0.0990
omp,Orthogonal Matching Pursuit,0.2448,0.1279,0.3558,0.8142,0.1713,1.2123,0.0350
dt,Decision Tree Regressor,0.1946,0.1448,0.3781,0.7879,0.1750,1.0139,0.0330


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

# Define X and Y 

In [118]:
x = scaled_df.drop(columns = ["Price(B)"])
y = scaled_df['Price(B)']

# Spliting X and Y into train and test and define Baysian Ridge model

In [119]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=47)
model = BayesianRidge()

In [120]:
cv = cross_val_score(model, X_train, y_train, cv = 10, scoring='r2')

In [121]:
model.fit(X_train, y_train)

BayesianRidge()

In [122]:
test_score = model.score(X_test, y_test)
print(f"Test Score (R-squared): {test_score}")

Test Score (R-squared): 0.8832699211613886


In [123]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.08266659062128451


# Model saved with pickle module

In [126]:
with open('baysian_ridge_model.pkl', 'wb') as file: pickle.dump(model, file)